In [2]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-02-05 14:15:12.539884: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 14:15:12.695692: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 14:15:13.580678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [4]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [5]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [6]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition import dataset
from swallowing_recognition import dence_net

In [7]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_movement_folder = directory_path / 'washino' / 'movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    

# 引数は(データ数，画像サイズ縦，画像サイズ横，色RGB，クラス数)
# train_data = dataset.DataSet(500, 224, 224, 3, 5)
# test_data = dataset.DataSet(70, 224, 224, 3, 5)

train_data = dataset.DataSet(800, 224, 224, 3, 2)
test_data = dataset.DataSet(28, 224, 224, 3, 2)

# train_data.folder_to_dataset(train_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# train_data.folder_to_dataset(train_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# train_data.folder_to_dataset(train_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# train_data.folder_to_dataset(train_movement_folder, np.array([0, 1, 0, 0, 0]), 3)
# train_data.folder_to_dataset(train_hitting_folder, np.array([1, 0, 0, 0, 0]), 4)

train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0)
train_data.folder_to_dataset(train_cough_folder, np.array(1), 400)
train_data.folder_to_dataset(train_voice_folder, np.array(1), 500)
train_data.folder_to_dataset(train_movement_folder, np.array(1), 600)
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 700)

# test_data.folder_to_dataset(test_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# test_data.folder_to_dataset(test_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# test_data.folder_to_dataset(test_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# test_data.folder_to_dataset(test_cough_folder, np.array([0, 0, 0, 1, 0]), 3)
# test_data.folder_to_dataset(test_swallowing_folder, np.array([0, 0, 1, 0, 0]), 4)

test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14)

In [14]:
# model = dence_net.DanceNet(5)

model = dence_net.DanceNet(2)

model_checkpoint = ModelCheckpoint(
    filepath='20240205_model_best.keras',  # 保存するファイルのパス
    monitor='val_loss',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='min'                 # 「減少」を監視
)

# 引数は(トレーニングデータ，ラベル，epochs，batch_size)
model.training(train_data.data, train_data.labels, 200, 32, model_checkpoint = model_checkpoint)
model.evaluate(test_data.data, test_data.labels)
model.save('20240205_binary_ep200_bs32.keras')

Epoch 1/200
23/23 [==============================] - ETA: 0s - loss: 0.3819 - accuracy: 0.8625
Epoch 1: val_loss improved from inf to 175.66191, saving model to 20240205_model_best.keras
23/23 [==============================] - 53s 210ms/step - loss: 0.3819 - accuracy: 0.8625 - val_loss: 175.6619 - val_accuracy: 0.0000e+00
Epoch 2/200
23/23 [==============================] - ETA: 0s - loss: 0.2447 - accuracy: 0.9111
Epoch 2: val_loss did not improve from 175.66191
23/23 [==============================] - 3s 113ms/step - loss: 0.2447 - accuracy: 0.9111 - val_loss: 195.5950 - val_accuracy: 0.0000e+00
Epoch 3/200
23/23 [==============================] - ETA: 0s - loss: 0.1603 - accuracy: 0.9444
Epoch 3: val_loss improved from 175.66191 to 63.27837, saving model to 20240205_model_best.keras
23/23 [==============================] - 3s 149ms/step - loss: 0.1603 - accuracy: 0.9444 - val_loss: 63.2784 - val_accuracy: 0.0000e+00
Epoch 4/200
23/23 [==============================] - ETA: 0s - los

Epoch 30/200
23/23 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 1.0000
Epoch 30: val_loss did not improve from 0.00714
23/23 [==============================] - 3s 110ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.4152 - val_accuracy: 0.8875
Epoch 31/200
23/23 [==============================] - ETA: 0s - loss: 0.0103 - accuracy: 0.9958
Epoch 31: val_loss did not improve from 0.00714
23/23 [==============================] - 2s 109ms/step - loss: 0.0103 - accuracy: 0.9958 - val_loss: 0.3383 - val_accuracy: 0.9250
Epoch 32/200
23/23 [==============================] - ETA: 0s - loss: 0.0136 - accuracy: 0.9944
Epoch 32: val_loss did not improve from 0.00714
23/23 [==============================] - 2s 107ms/step - loss: 0.0136 - accuracy: 0.9944 - val_loss: 0.8673 - val_accuracy: 0.8375
Epoch 33/200
23/23 [==============================] - ETA: 0s - loss: 0.0218 - accuracy: 0.9917
Epoch 33: val_loss did not improve from 0.00714
23/23 [========================

Epoch 60/200
23/23 [==============================] - ETA: 0s - loss: 0.0403 - accuracy: 0.9875
Epoch 60: val_loss did not improve from 0.00714
23/23 [==============================] - 3s 109ms/step - loss: 0.0403 - accuracy: 0.9875 - val_loss: 3.0878 - val_accuracy: 0.6250
Epoch 61/200
23/23 [==============================] - ETA: 0s - loss: 0.0318 - accuracy: 0.9917
Epoch 61: val_loss did not improve from 0.00714
23/23 [==============================] - 2s 108ms/step - loss: 0.0318 - accuracy: 0.9917 - val_loss: 0.6023 - val_accuracy: 0.8875
Epoch 62/200
23/23 [==============================] - ETA: 0s - loss: 0.0112 - accuracy: 0.9972
Epoch 62: val_loss improved from 0.00714 to 0.00125, saving model to 20240205_model_best.keras
23/23 [==============================] - 3s 144ms/step - loss: 0.0112 - accuracy: 0.9972 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 63/200
23/23 [==============================] - ETA: 0s - loss: 0.0212 - accuracy: 0.9931
Epoch 63: val_loss did not impro

23/23 [==============================] - 2s 106ms/step - loss: 1.2530e-04 - accuracy: 1.0000 - val_loss: 0.4599 - val_accuracy: 0.8750
Epoch 90/200
23/23 [==============================] - ETA: 0s - loss: 4.4407e-04 - accuracy: 1.0000
Epoch 90: val_loss did not improve from 0.00125
23/23 [==============================] - 2s 109ms/step - loss: 4.4407e-04 - accuracy: 1.0000 - val_loss: 0.4766 - val_accuracy: 0.8750
Epoch 91/200
23/23 [==============================] - ETA: 0s - loss: 8.5954e-05 - accuracy: 1.0000
Epoch 91: val_loss did not improve from 0.00125
23/23 [==============================] - 3s 109ms/step - loss: 8.5954e-05 - accuracy: 1.0000 - val_loss: 0.5710 - val_accuracy: 0.8500
Epoch 92/200
23/23 [==============================] - ETA: 0s - loss: 0.0261 - accuracy: 0.9958    
Epoch 92: val_loss did not improve from 0.00125
23/23 [==============================] - 2s 108ms/step - loss: 0.0261 - accuracy: 0.9958 - val_loss: 0.8078 - val_accuracy: 0.8625
Epoch 93/200
23/23 [

Epoch 119/200
23/23 [==============================] - ETA: 0s - loss: 7.6754e-04 - accuracy: 1.0000
Epoch 119: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 106ms/step - loss: 7.6754e-04 - accuracy: 1.0000 - val_loss: 0.8414 - val_accuracy: 0.8250
Epoch 120/200
23/23 [==============================] - ETA: 0s - loss: 5.2961e-04 - accuracy: 1.0000
Epoch 120: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 107ms/step - loss: 5.2961e-04 - accuracy: 1.0000 - val_loss: 0.6668 - val_accuracy: 0.8375
Epoch 121/200
23/23 [==============================] - ETA: 0s - loss: 2.7084e-04 - accuracy: 1.0000
Epoch 121: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 109ms/step - loss: 2.7084e-04 - accuracy: 1.0000 - val_loss: 0.5390 - val_accuracy: 0.8500
Epoch 122/200
23/23 [==============================] - ETA: 0s - loss: 1.1411e-04 - accuracy: 1.0000
Epoch 122: val_loss did not improve from 0.0

Epoch 148/200
23/23 [==============================] - ETA: 0s - loss: 6.3588e-05 - accuracy: 1.0000
Epoch 148: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 104ms/step - loss: 6.3588e-05 - accuracy: 1.0000 - val_loss: 0.4975 - val_accuracy: 0.9000
Epoch 149/200
23/23 [==============================] - ETA: 0s - loss: 5.2364e-05 - accuracy: 1.0000
Epoch 149: val_loss did not improve from 0.00000
23/23 [==============================] - 3s 109ms/step - loss: 5.2364e-05 - accuracy: 1.0000 - val_loss: 0.4937 - val_accuracy: 0.9000
Epoch 150/200
23/23 [==============================] - ETA: 0s - loss: 8.2657e-05 - accuracy: 1.0000
Epoch 150: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 104ms/step - loss: 8.2657e-05 - accuracy: 1.0000 - val_loss: 0.4994 - val_accuracy: 0.8875
Epoch 151/200
23/23 [==============================] - ETA: 0s - loss: 2.6184e-04 - accuracy: 1.0000
Epoch 151: val_loss did not improve from 0.0

23/23 [==============================] - 3s 111ms/step - loss: 0.0077 - accuracy: 0.9972 - val_loss: 0.5049 - val_accuracy: 0.8750
Epoch 178/200
23/23 [==============================] - ETA: 0s - loss: 0.0103 - accuracy: 0.9958
Epoch 178: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 109ms/step - loss: 0.0103 - accuracy: 0.9958 - val_loss: 0.8315 - val_accuracy: 0.8500
Epoch 179/200
23/23 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 1.0000
Epoch 179: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 108ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.6563 - val_accuracy: 0.8750
Epoch 180/200
23/23 [==============================] - ETA: 0s - loss: 5.3337e-04 - accuracy: 1.0000
Epoch 180: val_loss did not improve from 0.00000
23/23 [==============================] - 2s 106ms/step - loss: 5.3337e-04 - accuracy: 1.0000 - val_loss: 0.6224 - val_accuracy: 0.8750
Epoch 181/200
23/23 [=========

In [8]:
# model.evaluate_print()
test_data2 = dataset.DataSet(42, 224, 224, 3, 2)
test_data2.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data2.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data2.folder_to_dataset(test_voice_folder, np.array(1), 28)

In [9]:
model.evaluate(test_data2.data, test_data2.labels)

2/2 [==============================] - 0s 132ms/step - loss: 0.5006 - accuracy: 0.9286
Test accuracy:  0.9285714030265808
2/2 [==============================] - 0s 24ms/step
正しく分類されたサンプルのインデックス: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41]
誤って分類されたサンプルのインデックス: [ 4 17 36]
サンプル 4: 正解 = 0.0, 予測 = 1
サンプル 17: 正解 = 1.0, 予測 = 0
サンプル 36: 正解 = 1.0, 予測 = 0
